In [1]:
import pandas as pd
import geopandas as gpd
import shapely.geometry as geom
import numpy as np
import json

In [2]:

gdf : gpd.GeoDataFrame = gpd.read_file('local/MB_2021_AUST_SHP_GDA2020/MB_2021_AUST_GDA2020.shp')
# 4m

In [3]:
gdf : gpd.GeoDataFrame = gdf[['MB_CODE21', 'geometry']].rename(columns={'MB_CODE21': 'id'})

In [4]:
gdf_geom : gpd.GeoSeries = gdf.boundary
# 30s

In [5]:
# Combine multiple linestring and multilinestring geometries into single multilinestring geometry
gdml = gdf_geom.apply(lambda x: geom.MultiLineString([x]) if isinstance(x, geom.LineString) else x)
# 1m

In [6]:
gdl = gdml.apply(lambda x: x.geoms if x is not None else None).explode()
# 1m

In [7]:
gdl.dropna(inplace=True)

In [8]:
coords = gdl.apply(lambda x: np.array(x.coords))
# 1m

In [9]:
# Get all points
points_full = np.concatenate(coords.to_numpy())
# points_full.sort(axis=0)
len(points_full)
# 19647448
# 10s

19647448

In [10]:
# # Drop duplicates
# points = np.unique(points_full, axis=0)
# len(points)
# # 14983534

# # Split long and lat into separate columns, and save to csv, with index and header
# df = pd.DataFrame(points, columns=['long', 'lat'])
# df.to_csv('local/mb_points.csv', index=True, header=True)
# # 1m 30s

# pd.read_csv('local/mb_points.csv', index_col=0)
# # 10s

In [11]:
df_full = pd.DataFrame(points_full, columns=['long', 'lat'])
dfx = df_full.drop_duplicates().reset_index(drop=True)
# 20s

In [ ]:
# coords.apply(lambda line: [dfx[(dfx['long'] == x[0]) & (dfx['lat'] == x[1])].index[0] for x in line])
# # 0.5s per line -> 300,000 lines -> 48 hours

In [ ]:
[dfx[(dfx['long'] == x[0]) & (dfx['lat'] == x[1])].index[0] for x in coords[0]]

In [ ]:
coords[:5].apply(lambda line: [dfx[(dfx['long'] == x[0]) & (dfx['lat'] == x[1])].index[0] for x in line])

In [ ]:
dfx.set_index(['long', 'lat'], inplace=False, drop=False)
# 1m

In [14]:
dfx.reset_index(drop=False).rename(columns={'index': 'id'})

,id,long,lat
0,0,146.927851,-36.083648
1,1,146.928145,-36.082104
2,2,146.928340,-36.081122
3,3,146.929139,-36.081218
4,4,146.929059,-36.081643
...,...,...,...
10107713,10107713,96.887598,-12.111763
10107714,10107714,167.972812,-29.016558
10107715,10107715,167.972812,-29.015658
10107716,10107716,167.973840,-29.015658


In [19]:
df_lines = coords.to_frame(name='coords').reset_index(drop=False).rename(columns={'index': 'id'})

In [ ]:
# Explode the coords column into separate rows, also add order column
dfp = df_lines.explode('coords')
# 3m

In [26]:
sflines = gdl.apply(lambda x: np.array(x.coords.xy))
# 30s

In [45]:
dfli = coords.explode().to_frame(name='coords').reset_index(drop=False).rename(columns={'index': 'line_id'}).reset_index(drop=False).rename(columns={'index': 'point_order'})
# 2m 30s

In [47]:
dfli['long'] = dfli['coords'].apply(lambda x: x[0])
dfli['lat'] = dfli['coords'].apply(lambda x: x[1])
# 2m 30s

In [48]:
dfli.drop(columns=['coords'], inplace=True)
# 30s

In [50]:
dfli.to_csv('local/mb_lines.csv', index=False, header=True)
# 3m. 1GB

In [54]:
# Convert to base 64 and use only the last 5 characters
dfli['line_id'] = dfli['line_id'].apply(lambda x: x.to_bytes(4, 'big').hex()[-5:])
# 20s
# 40s -> MemoryError

MemoryError: Unable to allocate 300. MiB for an array with shape (19647448,) and data type complex128